## 1. Изучите структуру модуля src

In [1]:
import sys
PATH_TO_SRC = '..\\'
sys.path.append(PATH_TO_SRC)

# my functions
from src.myf import reduction_memory, create_module

import warnings
warnings.filterwarnings('ignore')

## 2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py

In [2]:
# Запишем prefilter_items с небольшой доработкой

pref = """

def prefilter_items(data, data_train):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5]['item_id'].tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01]['item_id'].tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    more_year = data[data['week_no'] > 52]['item_id'].tolist()
    data = data[~data['item_id'].isin(more_year)]
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    # Уберем слишком дорогие товары
    
    # ...
    
    print(data.shape[0])
    
    return data
    
"""

postf = """

def postfilter_items(user_id, recommednations):
    pass

"""

In [3]:
PATH_TO_UTILS = 'src\\utils.py'

create_module(
    path=(PATH_TO_SRC + PATH_TO_UTILS), 
    info=(pref + postf),
    f='w'
)# create_module

In [4]:
# Дополнительно дозапишу в utils train_test_split

tts = """

def train_test_split(data, to_split, test_size_weeks):
    
    data_ser = data[to_split]
    
    data_train = data[data_ser < data_ser.max() - test_size_weeks]
    data_test = data[data_ser >= data_ser.max() - test_size_weeks]

    return data_train, data_test

"""

create_module(
    path=(PATH_TO_SRC + PATH_TO_UTILS),
    info=tts,
    f='a'
)# create_module

## 3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py

In [5]:
gsir = '''

def get_similar_items_recommendation(item, model, itemid_to_id, id_to_itemid, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    
    res = [id_to_itemid[rec[0]] for rec in model.similar_items(
            
            itemid=itemid_to_id[item],
            N=N
        )# model similar_items
        ]# res list
    
    return res

'''

gsur = '''

def get_similar_users_recommendation(user, model, userid_to_id, id_to_itemid, user_item_matrix, N=5, filter_items=None):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    res = [id_to_itemid[rec[0]] for rec in model.recommend(
            
            userid=userid_to_id[user], 
            user_items=user_item_matrix,   # на вход user-item matrix
            N=N, 
            filter_already_liked_items=False, 
            filter_items=filter_items,  # !!! 
            recalculate_user=True
        
        )# model recommend
        ]# res list
    
    return res

'''

create_module(
    path=(PATH_TO_SRC + PATH_TO_UTILS),
    info=(gsir + gsur),
    f='a'
)# create_module

## 4. Создайте модуль src.recommenders.py.

In [6]:
PATH_TO_RECOMMENDERS = 'src\\recommender.py'

create_module(
    path=(PATH_TO_SRC + PATH_TO_RECOMMENDERS), 
    info='',
    f='w'
)# create_module

## Напищите код для класса ниже и положите его в src.recommenders.py

In [7]:
mainrec = '''

import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        user_item_matrix = pd.pivot_table(data=data, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробовать другие варианты
                                          aggfunc='count', 
                                          fill_value=0)
        
        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                        regularization=regularization,
                                        iterations=iterations,  
                                        num_threads=num_threads)
        
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, item, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        
        res = [self.id_to_itemid[rec[0]] for rec in self.model.similar_items(
            
            itemid=self.itemid_to_id[item],
            N=N
        )# model similar_items
        ]# res list

        assert len(res) == N, f'Количество рекомендаций != {N}'
        return res
    
    def get_similar_users_recommendation(self, user, N=5, filter_items=None):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        res = [self.id_to_itemid[rec[0]] for rec in self.model.recommend(
            
            userid=self.userid_to_id[user], 
            user_items=self.user_item_matrix,   # на вход user-item matrix
            N=N, 
            filter_already_liked_items=False, 
            filter_items=filter_items,  # !!! 
            recalculate_user=True
        
        )# model recommend
        ]# res list 

        assert len(res) == N, f'Количество рекомендаций != {N}'
        return res

'''

create_module(
    path=(PATH_TO_SRC + PATH_TO_RECOMMENDERS), 
    info=mainrec,
    f='a'
)# create_module

## 5. Проверьте, что все модули корректно импортируются

In [8]:
from src.utils import (
    
    train_test_split,
    
    prefilter_items, 
    postfilter_items,
 
    get_similar_users_recommendation, 
    get_similar_items_recommendation
    
) # src.utils

from src.metrics import (

    precision, precision_at_k, money_precision_at_k,
    hit_rate, hit_rate_at_k, 
    recall, recall_at_k,
    ap_k,
    reciprocal_rank

) # src.metrics

from src.recommender import MainRecommender

In [9]:
# для тестирования загружу df
import numpy as np
import pandas as pd

PATH_TO_RETAIL_TRAIN = '..\\datas\\retail_train.csv'

df = pd.read_csv(PATH_TO_RETAIL_TRAIN)
df = reduction_memory(df)

df.head()

before:		230.09 MB
after:		141.41 MB
reduсed:	88.68 MB


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [10]:
train, test = train_test_split(data=df, 
                               to_split='week_no', 
                               test_size_weeks=3)

In [11]:
filt_df = prefilter_items(data=df, 
                          data_train=train)
filt_df.head()

80555


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
10,2375,26984851516,1,9487839,1,2.00,364,-0.79,1642,1,0.0,0.0
13,1364,26984896261,1,920955,1,3.09,31742,0.00,1520,1,0.0,0.0
28,98,26984951769,1,1087347,1,0.25,337,0.00,1937,1,0.0,0.0
29,1172,26985025264,1,877180,1,2.29,396,0.00,946,1,0.0,0.0


In [12]:
recom = MainRecommender(data=filt_df)

In [13]:
sur = recom.get_similar_users_recommendation(user=98)
sur

[5570861, 822278, 923182, 9296994, 833904]

In [14]:
sir = recom.get_similar_items_recommendation(item=920955)
sir

[920955, 9194975, 998521, 1105489, 950560]

## 6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

In [27]:
responses = 1200000
interview = 120000
hired = 20000

In [30]:
def perсent(dividend, divider):
    return dividend / divider * 100

In [31]:
interview_chance = perсent(interview, responses)
interview_chance

10.0

In [33]:
hired_chance = perсent(hired, interview)
hired_chance

16.666666666666664

In [34]:
hired_chance_all = perсent(hired, responses)
hired_chance_all

1.6666666666666667

In [37]:
how_responses = 100 / hired_chance_all
how_responses

60.0